In [ ]:
pip install scikit-image

In [ ]:
import matplotlib.pyplot as plt
# %matplotlib inline
import numpy as np
import os
import shutil
import warnings
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import svm
from sklearn.metrics import confusion_matrix, f1_score, precision_score, \
                            recall_score, accuracy_score, classification_report

import seaborn as sns; sns.set()

from skimage import io, exposure, morphology, filters, color, \
                    segmentation, feature, measure, img_as_float, img_as_ubyte
from skimage.color.adapt_rgb import adapt_rgb, each_channel, hsv_value
warnings.simplefilter("ignore")
from scipy import ndimage as ndi

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, \
                         Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator

from keras.callbacks import Callback

dataset_default_path = './dataset'
train_set_path = './train'
test_set_path = './test'
weights_filename = "./alexnet_weights.h5"

rand_seed = 0

# Number of epochs, where each epoch represents one forward pass and one backward pass
# of all the training samples
num_epochs = 75
# the batch size defines the number of samples that will be propagated through the network.
# In other words it represents the number of training examples in one forward/backward pass.
# We will use the mini-batch approach instead of using the whole training set for each pass.
batch_size = 50
# shape of the dataset images: length = 450 px, height = 450 px, channels = 3 (RGB)
img_shape = (600, 450, 3)

# Minimum number of images of HAM10000 dataset to use 
min_samples = 200
# Maximum number of images (and their info) that will be printed during
# preprocessing, segmentation and feature extraction.
# This is needed to avoid the error message: 
# "Buffered data was truncated after reaching the output size limit"
# because there is a limited memory for displaying output of a cell on Colab
plot_limit = 200

In [ ]:
def image_show(image, size=None, cmap=None, no_axis=True):
    """
    Plot a single image.
    
    Parameters
    ----------
    image : ndarray
        Image to be plotted.
    size : tuple of in (length, height)
        Sizes of the figure.
    cmap : str
        Figure colormap.
    no_axis : bool
        Whether or not to show axis
    """
    if size is None:
        size = plt.rcParams['figure.figsize']
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=size)
    ax.grid(False)
    if cmap is not None:
        ax.imshow(image, cmap=cmap)
    else:
        ax.imshow(image)
    if no_axis:
        ax.axis('off')
    return fig, ax

In [ ]:
def imshow_all(*images, **kwargs):
    """
    Plot a series of images side-by-side.

    Convert all images to float so that images have a common intensity range.

    Parameters
    ----------
    limits : str
        Control the intensity limits. By default, 'image' is used set the
        min/max intensities to the min/max of all images. Setting `limits` to
        'dtype' can also be used if you want to preserve the image exposure.
    titles : list of str
        Titles for subplots. If the length of titles is less than the number
        of images, empty strings are appended.
    kwargs : dict
        Additional keyword-arguments passed to `imshow`.
    """
    images = [img_as_float(img) for img in images]

    titles = kwargs.pop('titles', [])
    if len(titles) != len(images):
        titles = list(titles) + [''] * (len(images) - len(titles))

    limits = kwargs.pop('limits', 'image')
    if limits == 'image':
        kwargs.setdefault('vmin', min(img.min() for img in images))
        kwargs.setdefault('vmax', max(img.max() for img in images))
    elif limits == 'dtype':
        vmin, vmax = dtype_limits(images[0])
        kwargs.setdefault('vmin', vmin)
        kwargs.setdefault('vmax', vmax)

    nrows, ncols = kwargs.get('shape', (1, len(images)))
    
    axes_off = kwargs.pop('axes_off', False)

    size = nrows * kwargs.pop('size', 5)
    width = size * len(images)
    if nrows > 1:
        width /= nrows * 1.33
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(width, size))
    for ax, img, label in zip(axes.ravel(), images, titles):
        ax.imshow(img, **kwargs)
        ax.set_title(label)
        ax.grid(False)
        if axes_off:
            ax.set_axis_off()

In [ ]:
def imshow_with_histogram(image, **kwargs):
    """
    Plot an image side-by-side with its histogram.

    - Plot the image next to the histogram
    - Plot each RGB channel separately (if input is color)
    - Automatically flatten channels
    - Select reasonable bins based on the image's dtype
    """
    # create a new subplot made of 2 figures
    width, height = plt.rcParams['figure.figsize']
    fig, (ax_image, ax_hist) = plt.subplots(ncols=2, figsize=(2*width, height))
    
    titles = kwargs.pop('titles', [])
    if len(titles) == 2:
        ax_image.set_title(titles[0])
        ax_hist.set_title(titles[1])

    # first plot the image
    kwargs.setdefault('cmap', plt.cm.gray)
    ax_image.imshow(image, **kwargs)
    
    # then plot its histogram
    if image.ndim == 2:
        hist, bin_centers = exposure.histogram(image)
        ax_hist.fill_between(bin_centers, hist, alpha=0.3, color='black', **kwargs)
    elif image.ndim == 3:
        # `channel` is the red, green, or blue channel of the image.
        for channel, channel_color in zip([channel for channel in np.rollaxis(image, -1)], 'rgb'):
            hist, bin_centers = exposure.histogram(channel)
            ax_hist.fill_between(bin_centers, hist, alpha=0.3, color=channel_color, **kwargs)
    ax_hist.set_xlabel('intensity')
    ax_hist.set_ylabel('# pixels')
    # ax_hist.grid(False)

    ax_image.set_axis_off()
    # match axes height
    plt.draw()
    dst = ax_hist.get_position()
    src = ax_image.get_position()
    ax_hist.set_position([dst.xmin, src.ymin, dst.width, src.height])
    return ax_image, ax_hist

In [ ]:
structuring_element = morphology.disk(7)

@adapt_rgb(each_channel)
def morph_closing_each(image, struct_element):
    return morphology.closing(image, struct_element)


@adapt_rgb(hsv_value)
def morph_closing_hsv(image, struct_element):
    return morphology.closing(image, struct_element)

In [ ]:
@adapt_rgb(each_channel)
def median_filter_each(image, struct_element):
    return filters.median(image, struct_element)


@adapt_rgb(hsv_value)
def median_filter_hsv(image, struct_element):
    return filters.median(image, struct_element)

In [ ]:
def build_train_test(num_samples=0, train_set_frac=0.75, dataset_path=dataset_default_path,
                     train_path=train_set_path, test_path=test_set_path,
                     overwrite=False):
    """
    Build the training and test set.

    Parameters
    ----------
    num_samples : int
        Number of samples to select from the dataset. If zero or less than zero,
        all the samples in the dataset will be chosen.
        If higher than the dataset size it will be set to the dataset size.
    train_set_frac : float
        It represents the proportion of the dataset to include in the train split.
        If less than 0.0 or greater than 1.0 it will be set to the default value (0.75).
    dataset_path : str
        It should point to a valid path containing the images of the HAM10000 dataset.
        If not valid it will be set to the default path './dataset'.
    train_path : str
        It should point to a valid path to store images of the training set.
        If not valid it will be set to the default path './train'.
    test_path : str
        It should point to a valid path to store images of the test set.
        If not valid it will be set to the default path './test'.
    overwrite : bool
        Specifies whether or not to overwrite the directories related to training
        and test set.
    
    Returns training set (DataFrame), test set (DataFrame) and class labels
    """
    # if 'dataset_path' is not valid set to the default path ('./dataset')
    if not os.path.isdir(dataset_path):
        print('Warning: {} not found. We will use {} for the dataset'.format(
              dataset_path, dataset_default_path))
        dataset_path = dataset_default_path
    # Get the absolute path of the dataset directory
    abs_dataset_path = os.path.abspath(dataset_path)

    # if 'train_path' is not valid set to the default path ('./train')
    if not os.path.isdir(train_path):
        print('Warning: {} not found. We will use {} to store the training set'.format(
              train_path, train_set_path))
        train_path = train_set_path
    # Get the absolute path of the training set
    abs_train_path = os.path.abspath(train_path)
    if overwrite:
        shutil.rmtree(abs_train_path)
        os.mkdir(abs_train_path)

    # If 'test_path' is not valid set to the default path ('./test')
    if not os.path.isdir(test_path):
        print('Warning: {} not found. We will use {} to store the test set'.format(
              test_path, test_set_path))
        test_path = test_set_path
    # Get the absolute path of the test set
    abs_test_path = os.path.abspath(test_path)
    if overwrite:
        shutil.rmtree(abs_test_path)
        os.mkdir(abs_test_path)

    # Read the dataset metadata from HAM10000_metadata.csv as a Pandas dataframe
    ham10000_df = pd.read_csv('HAM10000_metadata.csv')
    
    # For each image name in the 'image_id' column add the '.jpg' suffix.
    # This is needed by flow_from_dataframe since each item in 'image_id' column
    # must specify the (relative) path of the corresponding image
    # (see 'flow_from_dataframe' documentation)
    for img_idx in range(ham10000_df['image_id'].count()):
        ham10000_df.at[img_idx, 'image_id'] += '.jpg'

    # Get class labels
    classes = list(set(ham10000_df.dx))
    print('Class labels: {}'.format(classes))

    # Get and print the number of images in the dataset
    num_images = ham10000_df.image_id.count()
    print('Number of images in the HAM10000 dataset: {}'.format(num_images))

    if 0 < num_samples < min_samples:
        print('Warning: The chosen number of samples is low. '
              'It will be set by default to {}'.format(min_samples))
        ham10000_df = ham10000_df.sample(n=min_samples, random_state=rand_seed)
        print('Number of images in the reduced dataset: {}'.format(min_samples))
    elif num_samples > num_images:
        print('Warning: The chosen number of samples is greater than the dataset size. '
              'The dataset size will not be changed')
    elif min_samples <= num_samples <= num_images:
        ham10000_df = ham10000_df.sample(n=num_samples, random_state=rand_seed)
        print('Number of images in the reduced dataset: {}'.format(num_samples))

    # Print the number of images in each class (dataset)
    print('Number of images for each class (dataset):')
    print(ham10000_df.dx.value_counts())

    if train_set_frac < 0 or train_set_frac > 1.0:
        test_set_frac = 0.75

    # Randomly split the images into training and test set such that the
    # training set has 75% of the images of the dataset (and so the remaining
    # 25% of the dataset images will be put in the test set).
    # In addition, since the dataset is unbalanced (see the statistics printed above),
    # we use the 'stratify' parameter in order to split the dataset in a
    # stratified fashion (the training and test sets will contain the same proportion
    # of samples as the dataset for each class label)
    train_set, test_set = train_test_split(ham10000_df, train_size=train_set_frac,
                                           random_state=rand_seed,
                                           stratify=ham10000_df.dx)

    print('\nNumber of images in the training set : {}'.format(train_set.image_id.count()))
    print('Number of images for each class (training set):')
    print(train_set.dx.value_counts())
    print('\nNumber of images in the test set : {}'.format(test_set.image_id.count()))
    print('Number of images for each class (test set):')
    print(test_set.dx.value_counts())

    # Get the name of images in the training and test sets
    train_imgs = list(train_set['image_id'])
    test_imgs = list(test_set['image_id'])

    # Move the images designated for the training set into the related directory
    for img_name in train_imgs:
        src_path = os.path.join(abs_dataset_path, img_name)
        shutil.copy2(src_path, abs_train_path)

    # Move the images designated for the training set into the related directory
    for img_name in test_imgs:
        src_path = os.path.join(abs_dataset_path, img_name)
        shutil.copy2(src_path, abs_test_path)
  
    # Return the training set (DataFrame), the test set (DataFrame) and class labels
    return train_set, test_set, classes

In [ ]:
from skimage import img_as_ubyte
def preprocessing(train_path=train_set_path, test_path=test_set_path):
    """
    Apply some image processing techniques (CLAHE, morphology closing and median
    filter) to images of training and test sets.

    Parameters
    ----------
    train_path : str
        Training set path.
        If not valid it will be set to the default path './train'.
    test_path : str
        Test set path.
        If not valid it will be set to the default path './test'.
    
    Returns training set (DataFrame), test set (DataFrame) and class labels
    """

    print('-- PREPROCESSING --')

    # if 'train_path' is not valid set to the default path ('./train')
    if not os.path.isdir(train_path):
        train_path = train_set_path
    # get absolute path of the training set
    abs_train_path = os.path.abspath(train_path)

    # if 'test_path' is not valid set to the default path ('./test')
    if not os.path.isdir(test_path):
        test_path = test_set_path
    # get absolute path of the test set
    abs_test_path = os.path.abspath(test_path)

    # list of images in the training set
    img_train = [img_file for img_file in os.listdir(abs_train_path)
                 if os.path.isfile(os.path.join(abs_train_path, img_file)) and img_file.endswith('.jpg')]
    # list of images in the test set
    img_test = [img_file for img_file in os.listdir(abs_test_path)
                if os.path.isfile(os.path.join(abs_test_path, img_file)) and img_file.endswith('.jpg')]
    img_list = img_train + img_test

    for idx, image_name in enumerate(img_list):
        # get image path
        if image_name in img_train:
            image_path = os.path.join(abs_train_path, image_name)
        elif image_name in img_test:
            image_path = os.path.join(abs_test_path, image_name)
        else:
            print('Error: Cannot find {}'.format(image_name))
            return
        # read the image
        image = io.imread(image_path)

        if idx < plot_limit:
            # plot basic image info
            print('{:_<100}'.format(''))
            print('Image name: {}'.format(image_name))
            print('Image sizes: {}'.format(image.shape))
            print('Image data type: {}'.format(image.dtype))
        elif idx == plot_limit:
            print('Continuing image processing without printing the results ...')
    
        # 1) Equalize the image contrast using  the Contrast-Limited Adaptive 
        #    Histogram Equalization (CLAHE) method.
        equalized_adapthist = exposure.equalize_adapthist(image)
    
        # 2) Use morphological closing to remove hairs (in order to simulate the Razor filter).
        #    Images are processed using the 'morph_closing_each()' function which applies
        #    the morphological closing to each channel of the RGB image.
        img_morph_closing = morph_closing_each(equalized_adapthist, structuring_element)
    
        # 3) Use the median filter to apply interpolation on the obtained image.
        #    The median filter is implemented using the 'median_filter_each()'
        #    function which pass each of the RGB channels to the median filter
        #    one-by-one, and stitch the results back into an RGB image.
        img_filtered = median_filter_each(img_morph_closing, structuring_element)
    
        if idx < plot_limit:
            # plot the original image with its histogram
            imshow_with_histogram(image, titles=['Original image', 'Histogram'])

            # plot the image obtained after applying the CLAHE method
            imshow_with_histogram(equalized_adapthist,
                                  titles=['Image equalized (CLAHE)', 'Histogram'])

            # plot the images obtained using the morphological closing and the one
            # obtained after applying the median filter
            imshow_all(img_morph_closing, img_filtered, axes_off=True,
                       titles=['Morphological Closing', 'Median Filter'])
            plt.show();

        # save the final processed image
        io.imsave(image_path,img_as_ubyte(img_filtered))

In [ ]:
from skimage.segmentation import watershed
def images_segmentation(train_path=train_set_path, test_path=test_set_path):

    print('-- REGION-BASED SEGMENTATION --')
    
    # Python dictionaries containing the properties of the regions representing
    # the lesions (subdivided into training and test set)
    segmented_train_set = {}
    segmented_test_set = {}
    
    if not os.path.isdir(train_path):
        train_path = train_set_path
    # get absolute path of the training set
    abs_train_path = os.path.abspath(train_path)

    if not os.path.isdir(test_path):
        test_path = test_set_path
    # get absolute path of the test set
    abs_test_path = os.path.abspath(test_path)

    # list of images in the training set
    train_imgs = [img_file for img_file in os.listdir(abs_train_path)
                  if os.path.isfile(os.path.join(abs_train_path, img_file)) and img_file.endswith('.jpg')]
    # list of images in the test set
    test_imgs = [img_file for img_file in os.listdir(abs_test_path)
                 if os.path.isfile(os.path.join(abs_test_path, img_file)) and img_file.endswith('.jpg')]

    img_list = train_imgs + test_imgs

    for idx, image_name in enumerate(img_list):
        if idx < plot_limit:
            print('{:_<100}'.format(''))
            print('Image name: {}'.format(image_name))

        # get image path
        if image_name in train_imgs:
            image_path = os.path.join(abs_train_path, image_name)
        elif image_name in test_imgs:
            image_path = os.path.join(abs_test_path, image_name)
        else:
            print('Error: Cannot find {}'.format(image_name))
            return None, None
        # read image
        image = io.imread(image_path)
        # convert the original image into grayscale
        gray_img = color.rgb2gray(image)

        # 1] Apply Sobel filter
        elevation_map = filters.sobel(gray_img)

        # 2] Build image markers using the threshold obtained through the ISODATA filter
        markers = np.zeros_like(gray_img)
        threshold = filters.threshold_isodata(gray_img)
        markers[gray_img > threshold] = 1
        markers[gray_img < threshold] = 2
        
        # 3] Apply Wathershed algorithm in order to segment the image filtered
        #    using the markers
        segmented_img = watershed(elevation_map, markers)
        # 4] Improve segmantation:
        #    >  Fill small holes 
        segmented_img = ndi.binary_fill_holes(segmented_img - 1)
        #    > Remove small objects that have an area less than 800 px:
        #      this could be useful to exclude some regions that does not represent a lesion
        segmented_img = morphology.remove_small_objects(segmented_img, min_size=800)
        #    > Clear regions connected to the image borders.
        #      This operation is very useful when there are contour regions have a
        #      big area and so they can be exchanged with the lesion.
        #      However, this can also create some issues when the lesion region is
        #      connected to the image borders. In order to (try to) overcome this
        #      issue, we use a lesion identification algorithm (see below)
        img_border_cleared = segmentation.clear_border(segmented_img)

        # 5] Apply connected components labeling algorithm:
        #    it assigns labels to a pixel such that adjacent pixels of the same
        #    features are assigned the same label.
        # labeled_img, _ = ndi.label(segmented_img)
        labeled_img = morphology.label(img_border_cleared)

        if idx < plot_limit:
            # create a subplot of 3 figures in order to show elevation map,
            # markers and the segmanted image
            fig, ax = plt.subplots(1, 3, figsize=(10, 8))
            ax[0].imshow(elevation_map, cmap=plt.cm.gray)
            ax[0].set_title('elevation map')
            ax[0].set_axis_off()

            ax[1].imshow(markers, cmap=plt.cm.nipy_spectral)
            ax[1].set_title('markers')
            ax[1].set_axis_off()

            ax[2].imshow(segmented_img, cmap=plt.cm.gray)
            ax[2].set_title('segmentation')
            ax[2].set_axis_off()

            plt.tight_layout()
            plt.show();
        
        # 6] Lesion identification algorithm:
        # Compute properties of labeled image regions:
        # it will be used to automatically select the region that contains
        # the skin lesion according to area and extent
        props = measure.regionprops(labeled_img)
        # num labels -> num regions
        num_labels = len(props)
        # Get all the area of detected regions
        areas = [region.area for region in props]

        # If we have at least one region and the area of the region having the
        # biggest area is at least 1200 px, we choose it as the region that
        # contains the leson because if properly segmented (i.e., after removing
        # small objects and regions on the image contours (since in most of the
        # images, the lesion is in the center))
        if num_labels > 0 and areas[np.argmax(areas)] >= 1200:
            if idx < plot_limit:
                print('Num labels:', num_labels)
                print('Areas: {}'.format(areas))
            target_label = props[np.argmax(areas)].label
        else:
            # ... otherwise we could have one of the following two cases:
            # 1] num_labels == 0:
            #    this can happen when there is only one region (the one containing the lesion)
            #    but it has been deleted when applying the function segmentation.clear_border()
            # 2] num_labels > 0 but areas[np.argmax(areas)] < 1200:
            #    it means that there exists at least one region but all the regions have
            #    an area less than 1200 pixels.
            #    This can happen when the region containing the lesion is deleted
            #    with segmentation.clear_border() but there still other regions that
            #    could be "exhanged for" a lesion region
            #
            # Since both cases can be due to the deletion of the segmented area
            # because of the use of segmentation.clear_border(), the idea is to
            # backtrack to the original segmented image (the one obtained 
            # obtained before applying segmentation.clear_border()), apply again the
            # connected components labeling algorithm and extract the new region properties.
            # In addition, in order to find the region representing the lesion,
            # we use area and extent features by checking among the three largest regions
            # (if any because there could be only one or two regions) sorted in ascending order
            # (i.e. from the one having the largest area) the first that has an extent grater than 0.5.
            labeled_img = morphology.label(segmented_img)
            # Get new region properties
            props = measure.regionprops(labeled_img)
            # Get the new list of areas
            areas = [region.area for region in props]
            # List of regions' extent.
            # Each extent is defined as the ratio of pixels in the region  to pixels
            # in the total bounding box (computed as: area / (rows * cols))
            extents = [region.extent for region in props]
            if idx < plot_limit:
                print('Num labels: {}'.format(len(props)))
                print('Areas: {}'.format(areas))
                print('Extents: {}'.format(extents))
            # Get the index of the region having the largest area and if there are
            # more than one or two regions, find also the index of the second and
            # third most largest regions.
            region_max1 = np.argmax(areas)
            if len(props) > 1:
                areas_copy = areas.copy()
                areas_copy[region_max1] = 0
                region_max2 = np.argmax(areas_copy)
            if len(props) > 2:
                areas_copy[region_max2] = 0
                region_max3 = np.argmax(areas_copy)

            # If the largest region has an extent greater than 0.50, it is our target region
            if extents[region_max1] > 0.50:
                target_label = props[region_max1].label
            # ... else check if the extent of the second largest region is greater than 0.5,
            # and if so we have found our target region
            elif len(props) > 1 and extents[region_max2] > 0.50:
                target_label = props[region_max2].label
            # ... else if the third largest region has an extent greater than 0.50,
            # it is (more probably) the one containing the lesion
            elif len(props) > 2 and extents[region_max3] > 0.50:
                target_label = props[region_max3].label
            # ... otherwise we choose the largest region
            else:
                target_label = props[region_max1].label

            # NOTE: another possible approarch could be to select as the target region
            #       the one having the largest extent among the 3 largest regions found:
            # if len(props) > 2:
            #     extents_largest_reg = [val if idx in (region_max1, region_max2, region_max3) else 0.0
            #                            for idx, val in enumerate(extents)]
            # elif len(props) > 1:
            #     extents_largest_reg = [val if idx in (region_max1, region_max2) else 0.0
            #                            for idx, val in enumerate(extents)]
            # else:
            #     extents_largest_reg = [val if idx in (region_max1,) else 0.0
            #                            for idx, val in enumerate(extents)]
            # target_label = props[np.argmax(extents_largest_reg)].label

        # assign label 0 to all the pixels that are not in the target region (that is
        # the ragion that more probably contains the lesion)
        for row, col in np.ndindex(labeled_img.shape):
            if labeled_img[row, col] != target_label:
                labeled_img[row, col] = 0
        # Convert the labeled image into its RGB version
        image_label_overlay = color.label2rgb(labeled_img, gray_img)

        if idx < plot_limit:
            print('Chosen label: {}'.format(target_label))
            # Plot the original image ('image') in which the contours of all the
            # segmented regions are highlighted
            fig, axes = plt.subplots(1, 2, figsize=(8, 6), sharey=True)
            axes[0].imshow(image)
            axes[0].contour(segmented_img, [0.5], linewidths=1.2, colors='y')
            axes[0].axis('off')
            # Plot 'image_label_overlay' that contains the target region highlighted
            axes[1].imshow(image_label_overlay)
            axes[1].axis('off')

            plt.tight_layout()
            plt.show();
        elif idx == plot_limit:
            print('Continuing segmentation without printing the results ...')

        # Add the the found region into the proper dictionary according to whether
        # the current image belongs to the training or the test set
        if image_name in train_imgs:
            segmented_train_set[image_name] = props[target_label - 1]
        elif image_name in test_imgs:
            segmented_test_set[image_name] = props[target_label - 1]

    return segmented_train_set, segmented_test_set

In [ ]:
def features_extraction(segmented_region_train, segmented_region_test, out_df=False,
                        train_path=train_set_path, test_path=test_set_path):

    print('-- FEATURE EXTRACTION --')

    if out_df:
        train_list = []
        test_list = []
        train_index = []
        test_index = []
    else:
        train = {}
        test = {}

    if not os.path.isdir(train_path):
        train_path = train_set_path
    # absolute path of the training set
    abs_train_path = os.path.abspath(train_path)
  
    # absolute path of the test set
    if not os.path.isdir(test_path):
        test_path = test_set_path
    abs_test_path = os.path.abspath(test_path)

    segmented_regions = {**segmented_region_train, **segmented_region_test}  # Python 3.5
    # segmented_regions = {k: v for d in (segmented_region_train, segmented_region_test) for k, v in d.items()}
    for idx, image_name in enumerate(segmented_regions):
        if idx < plot_limit:
            print('{:_<100}'.format(''))
            print('Image name: {}'.format(image_name))
        elif idx == plot_limit:
            print('\nContinuing feature extraction without printing the results ...')

        if image_name in segmented_region_train:
            image_path = os.path.join(abs_train_path, image_name)
        elif image_name in segmented_region_test:
            image_path = os.path.join(abs_test_path, image_name)
        else:
            print('Error: Cannot find {}'.format(image_name))
            return None

        image = io.imread(image_path)
        gray_img = color.rgb2gray(image)

        lesion_region = segmented_regions[image_name]

        # 1] ASYMMETRY
        area_total = lesion_region.area
        img_mask = lesion_region.image

        horizontal_flip = np.fliplr(img_mask)
        diff_horizontal = img_mask * ~horizontal_flip

        vertical_flip = np.flipud(img_mask)
        diff_vertical = img_mask * ~vertical_flip

        diff_horizontal_area = np.count_nonzero(diff_horizontal)
        diff_vertical_area = np.count_nonzero(diff_vertical)
        asymm_idx = 0.5 * ((diff_horizontal_area / area_total) + (diff_vertical_area / area_total))
        ecc = lesion_region.eccentricity
        # mmr = lesion_region.minor_axis_length / lesion_region.major_axis_length

        if idx < plot_limit:
            print('-- ASYMMETRY --')
            print('Diff area horizontal: {:.3f}'.format(np.count_nonzero(diff_horizontal)))
            print('Diff area vertical: {:.3f}'.format(np.count_nonzero(diff_vertical)))
            print('Asymmetric Index: {:.3f}'.format(asymm_idx))
            print('Eccentricity: {:.3f}'.format(ecc))
            # print('Minor-Major axis ratio: {}'.format(mmr))
        
            imshow_all(img_mask, horizontal_flip, diff_horizontal,
                       titles=['image mask', 'horizontal flip', 'difference'], size=4, cmap='gray')
            imshow_all(img_mask, vertical_flip, diff_vertical,
                       titles=['image mask', 'vertical flip', 'difference'], size=4, cmap='gray')
            plt.show();

        # 2] Border irregularity:
        compact_index = (lesion_region.perimeter ** 2) / (4 * np.pi * area_total)
        if idx < plot_limit:
            print('\n-- BORDER IRREGULARITY --')
            print('Compact Index: {:.3f}'.format(compact_index))

        # 3] Color variegation:
        sliced = image[lesion_region.slice]
        lesion_r = sliced[:, :, 0]
        lesion_g = sliced[:, :, 1]
        lesion_b = sliced[:, :, 2]

        C_r = np.std(lesion_r) / np.max(lesion_r)
        C_g = np.std(lesion_g) / np.max(lesion_g)
        C_b = np.std(lesion_b) / np.max(lesion_b)

        if idx < plot_limit:
            print('\n-- COLOR VARIEGATION --')
            print('Red Std Deviation: {:.3f}'.format(C_r))
            print('Green Std Deviation: {:.3f}'.format(C_g))
            print('Blue Std Deviation: {:.3f}'.format(C_b))
            # imshow_all(lesion_r, lesion_g, lesion_b)
            # plt.show();

        # Alternative method to compute colorfulness:
        # https://www.pyimagesearch.com/2017/06/05/computing-image-colorfulness-with-opencv-and-python/
        # compute rg = Red - Green
        # rg = np.absolute(lesion_r - lesion_g)
        # compute yb = 0.5 * (Red + Green) - Blue
        # yb = np.absolute(0.5 * (lesion_r + lesion_g) - lesion_b)
        #
        # compute the mean and standard deviation of both `rg` and `yb`
        # (rb_mean, rb_std) = (np.mean(rg), np.std(rg))
        # (yb_mean, yb_std) = (np.mean(yb), np.std(yb))
        #
        # combine the mean and standard deviations
        # std_root = np.sqrt((rb_std ** 2) + (yb_std ** 2))
        # mean_root = np.sqrt((rb_mean ** 2) + (yb_mean ** 2))
        #
        # derive the "colorfulness" metric (color index)
        # color_index = std_root + (0.3 * mean_root)

        # 4] Diameter:
        eq_diameter = lesion_region.equivalent_diameter
        if idx < plot_limit:
            print('\n-- DIAMETER --')
            print('Equivalent diameter: {:.3f}'.format(eq_diameter))
            # optionally convert the diameter in mm, knowing that 1 px = 0.265 mm:
            # 1 px : 0.265 mm = diam_px : diam_mm -> diam_mm = diam_px * 0.265
            print('Diameter (mm): {:.3f}'.format(eq_diameter * 0.265))

        # 5] Texture:
        glcm = feature.greycomatrix(image=img_as_ubyte(gray_img), distances=[1],
                                    angles=[0, np.pi/4, np.pi/2, np.pi * 3/2],
                                    symmetric=True, normed=True)

        correlation = np.mean(feature.greycoprops(glcm, prop='correlation'))
        homogeneity = np.mean(feature.greycoprops(glcm, prop='homogeneity'))
        energy = np.mean(feature.greycoprops(glcm, prop='energy'))
        contrast = np.mean(feature.greycoprops(glcm, prop='contrast'))
        dissimilarity = np.mean(feature.graycoprops(glcm, 'dissimilarity'))
        ASM=np.mean(feature.graycoprops(glcm, 'ASM'))
        

        if idx < plot_limit:
            print('\n-- TEXTURE --')
            print('Correlation: {:.3f}'.format(correlation))
            print('Homogeneity: {:.3f}'.format(homogeneity))
            print('Energy: {:.3f}'.format(energy))
            print('Contrast: {:.3f}'.format(contrast))
            print('Dissimilarity: {:.3f}'.format(dissimilarity))

        if image_name in segmented_region_train:
            if out_df:
                data_list = train_list
                train_index.append(image_name.split('.')[0])
            else:
                dataset = train
        elif image_name in segmented_region_test:
            if out_df:
                data_list = test_list
                test_index.append(image_name.split('.')[0])
            else:
                dataset = test
        else:
            print('Error: Cannot find {}'.format(image_name))
            return None
        
        if out_df:
            data_list.append([asymm_idx, ecc, compact_index, C_r, C_g, C_b,
                              eq_diameter, correlation, homogeneity, energy, contrast,dissimilarity,ASM,localization])
        else:
            dataset[image_name] = [asymm_idx, ecc, compact_index, C_r, C_g, C_b,
                                   eq_diameter, correlation, homogeneity, energy, contrast,dissimilarity,ASM,localization]

    if out_df:
        attr = ['AsymIdx', 'Eccentricity', 'CI', 'StdR', 'StdG', 'StdB', 
                'Diameter', 'Correlation', 'Homogeneity', 'Energy', 'Contrast',"Dissimilarity","ASM","localization"]
        train_df = pd.DataFrame(data=train_list, index=train_index, columns=attr)
        test_df = pd.DataFrame(data=test_list, index=test_index, columns=attr)
        return train_df, test_df
    else:
        return train, test

In [ ]:
train_df, test_df, class_labels = build_train_test(num_samples=10015,
                                                   overwrite=True)

In [ ]:
preprocessing()

In [ ]:
segmented_region_train, segmented_region_test = images_segmentation()

In [ ]:
# dictionary version:
features_train, features_test = features_extraction(segmented_region_train, segmented_region_test)

train_names = list(features_train.keys())
X_train = list(features_train.values())
y_train = [train_df[train_df['image_id'] == img_name].reset_index().at[0, 'dx'] for img_name in train_names]

test_names = list(features_test.keys())
X_test = list(features_test.values())
y_test = [test_df[test_df['image_id'] == img_name].reset_index().at[0, 'dx'] for img_name in test_names]

# DataFrame version:
#train_features_df, test_features_df = feature_extraction(segmented_region_train,
                                                          #segmented_region_test,
                                                          #out_df=True)
# X_train = [list(train_features_df.iloc[row_idx])
#            for row_idx in range(train_features_df.image_id.count())]
# y_train = train_features_df.loc[:, 'dx']
#
# X_test = [list(test_features_df.iloc[row_idx])
#           for row_idx in range(test_features_df.image_id.count())]
# y_test = test_features_df.loc[:, 'dx']

In [ ]:
features_train

In [ ]:
# clf = svm.SVC(gamma='scale', class_weight='balanced')
# clf.fit(X_train, y_train)
# y_pred = clf.predict(X_test)

print("Training the SVM classifier...")

param_grid = {'C': [1, 1e1, 1e2, 1e3, 5e3, 1e4],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],
              'class_weight': [None, 'balanced']}
clf = GridSearchCV(svm.SVC(kernel='rbf'), param_grid)
clf = clf.fit(X_train, y_train)

print("Best estimator found by Grid Search:")
print(clf.best_estimator_)

y_pred = clf.predict(X_test)

In [ ]:
print('*** TEST SET PERFORMANCE EVALUATION - Segmentation + Feature Extraction + SVM ***')
# compute and plot performance metrics
accuracy = accuracy_score(y_test, y_pred)
val_f1 = f1_score(y_test, y_pred, average='weighted')
val_recall = recall_score(y_test, y_pred, average='weighted')
val_precision = precision_score(y_test, y_pred, average='weighted')

print('Accuracy: {:.3f}'.format(accuracy))
print('F1-score: {:.3f}'.format(val_f1))
print('Recall: {:.3f}'.format(val_recall))
print('Precision: {:.3f}'.format(val_precision))

print('\nClassification report:')
print(classification_report(y_test, y_pred, target_names=class_labels))

plot_confusion_matrix(y_test, y_pred, class_labels)

In [ ]:

trainimgid=[]
trainentries=[]
testimgid=[]
testentries=[]
for k,v in features_train.items():
    x=[]
    x.append(k)
    for i in v:
        x.append(i)
    trainentries.append(x)
for k,v in features_test.items():
    x=[]
    x.append(k)
    for i in v:
        x.append(i)
    testentries.append(x)
    

In [ ]:
features=[['ImageID','AsymIdx', 'Eccentricity', 'CI', 'StdR', 'StdG', 'StdB', 
                'Diameter', 'Correlation', 'Homogeneity', 'Energy', 'Contrast',"Dissimilarity","ASM"]]

In [ ]:
import csv
with open('train.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(features)
    writer.writerows(trainentries)
with open('test.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(features)
    writer.writerows(testentries)
df = pd.read_csv("train.csv")
df["Cancer_type"] = y_train
df.to_csv("train.csv", index=False)
df = pd.read_csv("test.csv")
df["Cancer_type"] = y_test
df.to_csv("test.csv", index=False)


In [ ]:
train_names = list(features_train.keys())
X_train = list(features_train.values())
y_train = [train_df[train_df['image_id'] == img_name].reset_index().at[0, 'dx'] for img_name in train_names]

test_names = list(features_test.keys())
X_test = list(features_test.values())
y_test = [test_df[test_df['image_id'] == img_name].reset_index().at[0, 'dx'] for img_name in test_names]

In [ ]:
datagen = ImageDataGenerator()